In [3]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.models import load_model
from src.dataset_builders.text_data import create_easy_token_extraction_data
from src.nn_settings.simple_gru_config import Config
from src.simple_rnns import simple_gru, mikes_amount_rnn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
config = Config()
data = config.data
num_input_characters = config.num_input_characters
num_output_characters = config.num_output_characters
input_encoding = config.input_encoding
output_encoding = config.output_encoding
enc = config.encoder
max_output_length = enc.max_output_length

print("input chars", num_input_characters)
print("data size", len(data))
print(data[:5])


Creating Data set


/Users/i862304/workspace/NeuralNetworkPlayground/notebooks/src/dataset_builders/text_data.py:59: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  sample_data = np.vstack((get_one_sample() for _ in range(data_size)))


Building Vocabulary
Building Encoding
Setting Attributes
Done
input chars 9
data size 50000
[['dbXedYee' 'ed']
 ['adXbeYaa' 'be']
 ['bdXedYbd' 'ed']
 ['baXdbYae' 'db']
 ['ddXeaYdb' 'ea']]


In [7]:
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, GRU, Dense, LSTM
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras.layers import TimeDistributed, Bidirectional, RepeatVector
from tensorflow.python.keras.optimizers import RMSprop

def temp_gru(input_size, 
             output_size,
             latent_dim,
             max_input_length, 
             max_out_seq_len):
    """
    A simple GRU stripped down about as much as possible
    """
    inputs = Input(shape=(max_input_length, input_size,))
    x = GRU(latent_dim,
            return_sequences=False,
            )(inputs)
    x = RepeatVector(max_out_seq_len)(x)
    x = Dense(output_size, activation="softmax")(x)
    outputs = TimeDistributed()(x)
    model = Model(inputs, outputs)

#     rms_prop = RMSprop(
#                        rho=0.9,
#                        decay=0.0,
#                        epsilon=1e-08,
#                        )
#     model.compile(loss="categorical_crossentropy",
#                   optimizer=rms_prop)
    return model


latent_dim = 16
model = temp_gru(num_input_characters,
                num_output_characters,
                latent_dim,
                max_input_length, 
                max_output_length,)



rms_prop = RMSprop(
                   rho=0.9,
                   decay=0.0,
                   epsilon=1e-08,
                   )
model.compile(loss="categorical_crossentropy",
              optimizer=rms_prop)


TypeError: temp_gru() missing 1 required positional argument: 'max_out_seq_len'

In [12]:
batch_size = 32
epochs = 50
latent_dim = 128
save_path = "../data/keras_gru_token_extraction_2.h5"

if not os.path.isfile(save_path):
    print("Creating Model")
    model = simple_gru(num_input_characters,
               num_output_characters,
               latent_dim,
               max_output_length,
              )
else:
    print("Loading Model")
    model = load_model(save_path)

# define the checkpoint
checkpoint = ModelCheckpoint(save_path, 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')
callbacks_list = [checkpoint]

print(model.summary())

Loading Model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 128)               52992     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 2, 128)            0         
_________________________________________________________________
time_distributed (TimeDistri (None, 2, 9)              1161      
Total params: 54,153
Trainable params: 54,153
Non-trainable params: 0
_________________________________________________________________
None


In [75]:
# fit the model
model.fit(input_encoding, output_encoding,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1)
model.save(save_path)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 25s 553us/step - loss: 0.5538 - val_loss: 0.5551
Epoch 2/50
45000/45000 [==============================] - 25s 556us/step - loss: 0.5538 - val_loss: 0.5554
Epoch 3/50
45000/45000 [==============================] - 25s 558us/step - loss: 0.5538 - val_loss: 0.5551
Epoch 4/50
45000/45000 [==============================] - 25s 558us/step - loss: 0.5538 - val_loss: 0.5550
Epoch 5/50
45000/45000 [==============================] - 25s 559us/step - loss: 0.5538 - val_loss: 0.5549
Epoch 6/50
45000/45000 [==============================] - 25s 558us/step - loss: 0.5538 - val_loss: 0.5553
Epoch 7/50
45000/45000 [==============================] - 25s 564us/step - loss: 0.5538 - val_loss: 0.5552
Epoch 8/50
45000/45000 [==============================] - 25s 557us/step - loss: 0.5538 - val_loss: 0.5551
Epoch 9/50
45000/45000 [==============================] - 25s 566us/step - loss: 0.5538 - val_l

## Try Mike's Amount RNN

In [ ]:
# use_embedding = False
# max_out_seq_len = 10
# hidden_size = 32
# # build_model(len(rcpt_vocabulary), len(amt_vocabulary), MAX_DIGITS, 128, embedding=embedding)
# model = mikes_amount_rnn(num_input_characters, num_output_characters, 
#                          max_out_seq_len=max_out_seq_len, hidden_size=hidden_size, 
#                          embedding=use_embedding)

# Test Encoder

In [ ]:
print(input_encoding.shape)
print(output_encoding.shape)

N = 43
print(enc.decode(np.array([input_encoding[43]])))
print(enc.decode(np.array([output_encoding[43]])))

# Test Model

In [14]:
# test_str = "mqjrzvoXcorgiYaq snxoqkswccsaaahivlq"
test_str = "aaXcaYbc"
test_enc = enc.encode([test_str], )

test_prediction = model.predict(test_enc)
# print(test_prediction)
test_result = enc.decode(test_prediction)
test_result

array([b'cc'], dtype='|S8')

# roll output data for teacher forcing

In [ ]:
# decoder_input_data is a 3D array of shape (num_pairs, max_french_sentence_length, num_french_characters) 
# containg a one-hot vectorization of the French sentences.
# decoder_target_data is the same as decoder_input_data but offset by one timestep. 
# decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :].


# temp_enc = Encoder(vocabulary_map)
# temp = np.arange(0, 30).reshape((5, 3, 2))
# temp

In [ ]:
# n=0
# print("target")
# print(temp[:, n, :])
# print("input")
# print(temp[:, n+1, :])